# Hyper-parameter optimization

In [ ]:
!pip install hyperas proglog

In [ ]:
import proglog
proglog.notebook()

In [9]:

import time
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import multi_gpu_model
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice

def norm(data):
    height= 375
    width = 1242
    
    data.loc[:,'xmin'] = data['xmin'] / width 
    data.loc[:,'xmax'] = data['xmax'] / width
    data.loc[:,'ymax'],data.loc[:,'ymin']  = (height - data['ymin']) / height , (height - data['ymax']) / height
    
    return data

def data():
    # ----------- import data and scaling ----------- #
    df_train = pd.read_csv('data/train.csv')
    df_test = pd.read_csv('data/test.csv')

    X_train = df_train[['ymin','xmin','ymax', 'xmax']]
    y_train = df_train[['xloc','zloc']].values

    X_test = df_test[['ymin','xmin','ymax', 'xmax']]
    y_test = df_test[['xloc','zloc']].values

    X_train = norm(X_train).values
    X_test = norm(X_test).values

    return X_train, y_train, X_test, y_test

def create_model(X_train, y_train, X_test, y_test):
    # ----------- define model ----------- #
    model = Sequential()

    model.add(Dense({{choice([10,50,100])}}, input_shape=(4,)))

    model.add(Dense({{choice([10,50,100])}}, activation='relu'))
    model.add(Dense({{choice([10,50,100])}}, activation='relu'))
   

    model.add(Dense(2))

#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='mean_squared_error', metrics=['mae'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    # ----------- define callbacks ----------- #
    earlyStopping = EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=7,
								       verbose=1, epsilon=1e-5, mode='min')
#     tensorboard = TensorBoard(log_dir="logs/model@{}".format(int(time.time())))

    # ----------- start training ----------- #
    model.fit(X_train, y_train,
              batch_size={{choice([1024, 2048])}},
              epochs=2000,
#               callbacks=[tensorboard],
              verbose=0,
              validation_split=0.2)

    # ----------- evaluate model ----------- #
    score, acc = model.evaluate(X_test, y_test, verbose=1)
    print('Test accuracy:', score)

    # ----------- save model and weights ----------- #

    return {'loss': score, 'status': STATUS_OK, 'model': model}


trials = Trials()
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=3,
                                      trials=trials,
                                      eval_space=True,
                                      functions=[norm],
                                     notebook_name='hyperas')

X_train, Y_train, X_test, Y_test = data()
print("Evaluation of best performing model:", best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)


>>> Imports:
#coding=utf-8

try:
    import proglog
except:
    pass

try:
    import time
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from tensorflow.keras.utils import multi_gpu_model
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [10,50,100]),
        'Dense_1': hp.choice('Dense_1', [10,50,100]),
        'Dense_2': 

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


  1/128 [..............................]             
 - ETA: 0s - loss: 204.5824 - mae: 10.4179           
                                                    
 34/128 [======>.......................]             
 - ETA: 0s - loss: 186.3542 - mae: 10.0413           
                                                    
 64/128 [==============>...............]             
 - ETA: 0s - loss: 182.2450 - mae: 9.9436            
                                                     
 95/128 [=====================>........]             
 - ETA: 0s - loss: 180.2037 - mae: 9.8998            
                                                     
126/128 [============================>.]             
 - ETA: 0s - loss: 179.7433 - mae: 9.8765            
                                                     
128/128 [==============================]             
 - 0s 2ms/step - loss: 179.7898 - mae: 9.8735        

Test accuracy:                                       
179.7898406982422            

In [ ]:
print(keras.__version__)

In [8]:
print(best_run)


{'Dense': 50, 'Dense_1': 10, 'Dense_2': 100, 'batch_size': 1024, 'optimizer': 'adam'}
